# Semantic Memory Session Manager

This notebook goes over our Semantic Memory Session Manager, refered to in the code base as `SemanticSessionManager`.
The goal of this class is to wrap Semantic Memory with user permissions and data isolation.

## Initialize resources

In [1]:
# Imports
import os

import openai
from sqlalchemy.engine.url import URL
from sqlalchemy.ext.asyncio import create_async_engine

from memmachine.common.embedder.openai_embedder import (
    OpenAIEmbedder,
    OpenAIEmbedderParams,
)
from memmachine.common.language_model.openai_language_model import OpenAILanguageModel
from memmachine.semantic_memory.semantic_memory import SemanticService
from memmachine.semantic_memory.semantic_model import Resources
from memmachine.semantic_memory.semantic_session_manager import SemanticSessionManager
from memmachine.semantic_memory.semantic_session_resource import (
    ALL_MEMORY_TYPES,
    IsolationType,
    SessionIdManager,
    SessionResourceRetriever,
)
from memmachine.semantic_memory.storage.sqlalchemy_pgvector_semantic import (
    SqlAlchemyPgVectorSemanticStorage,
)
from memmachine.server.prompt.financial_analyst_prompt import (
    FinancialAnalystSemanticCategory,
)
from memmachine.server.prompt.health_assistant_prompt import (
    HealthAssistantSemanticCategory,
)
from memmachine.server.prompt.profile_prompt import UserProfileSemanticCategory
from memmachine.server.prompt.writing_assistant_prompt import (
    WritingAssistantSemanticCategory,
)

In [2]:
## If using `.env` file

from dotenv import load_dotenv

load_dotenv()

True

### AI and database

In [21]:
openai_api_key = os.getenv("OPENAI_API_KEY")


model = OpenAILanguageModel(
    {
        "api_key": openai_api_key,
        "model": "gpt-4o-mini",
    }
)

client = openai.AsyncOpenAI(api_key=openai_api_key)
embedder = OpenAIEmbedder(
    OpenAIEmbedderParams(
        client=client,
        model="text-embedding-3-small",
        dimensions=1536,
    )
)


host = os.environ.get("POSTGRES_HOST")
port = os.environ.get("POSTGRES_PORT")
user = os.environ.get("POSTGRES_USER")
password = os.environ.get("POSTGRES_PASSWORD")
database = os.environ.get("POSTGRES_DB")

engine = create_async_engine(
    URL.create(
        "postgresql+asyncpg",
        username=user,
        password=password,
        host=host,
        port=port,
        database=database,
    )
)

storage_client = SqlAlchemyPgVectorSemanticStorage(engine)

In [22]:
await storage_client.startup()

### Semantic Memory categories

In [23]:
user_knowledge = [
    UserProfileSemanticCategory,
    WritingAssistantSemanticCategory,
    HealthAssistantSemanticCategory,
]

role_knowledge = [
    UserProfileSemanticCategory,
]

product_knowledge = [FinancialAnalystSemanticCategory]

### Semantic Service

In [24]:
default_resources = {
    IsolationType.USER: Resources(
        embedder=embedder,
        language_model=model,
        semantic_categories=user_knowledge,
    ),
    IsolationType.ROLE: Resources(
        embedder=embedder,
        language_model=model,
        semantic_categories=role_knowledge,
    ),
    IsolationType.SESSION: Resources(
        embedder=embedder,
        language_model=model,
        semantic_categories=product_knowledge,
    ),
}

simple_session_id_manager = SessionIdManager()

resource_retriever = SessionResourceRetriever(
    simple_session_id_manager, default_resources
)

semantic_service = SemanticService(
    SemanticService.Params(
        resource_retriever=resource_retriever,
        semantic_storage=storage_client,
        feature_update_interval_sec=0.05,
        feature_update_message_limit=10,
        feature_update_time_limit_sec=0.05,
    )
)

semantic_session_manager = SemanticSessionManager(
    semantic_service=semantic_service,
    history_storage=storage_client,
)

## Usage

### Creating a session

We are going to start by creating a model session for a user.

In [25]:
user_a_session = simple_session_id_manager.generate_session_data(
    user_id="user_a_profile",
    role_id="finance_manager",
    session_id="hq_office",
)

### Adding a memory to your user profile

In [35]:
feature_a_id = await semantic_session_manager.add_feature(
    user_a_session,
    memory_type=IsolationType.USER,
    category_name="personal_profile",
    tag="food_and_cuisine",
    feature="favorite_pizza",
    value="peperoni_pizza",
)

feature_a_id

2

### Searching memories in your profile

In [36]:
features = await semantic_session_manager.get_set_features(
    user_a_session,
    memory_type=ALL_MEMORY_TYPES,
    tag_names=["food_and_cuisine", "hobbies"],
)

features

[SemanticFeature(set_id='mem_user_user_a_profile', category='personal_profile', tag='food_and_cuisine', feature_name='favorite_pizza', value='peperoni_pizza', metadata=Metadata(citations=None, id=2, other=None))]

When searching, we form an `and` between the different search criteria, and an `or` inside each criteria.

```
(
    memory_type = IsolationType.USER OR IsolationType.ROLE
    AND
    category_name = "personal_profile" OR "health_profile"
    AND
    tag_name = "food_and_cuisine" OR "hobbies"
)
```


In [37]:
await semantic_session_manager.get_set_features(
    user_a_session,
    memory_type=[IsolationType.USER, IsolationType.ROLE],
    category_names=["personal_profile", "health_profile", "writing_profile"],
    tag_names=["food_and_cuisine", "hobbies"],
    feature_names=["favorite_pizza", "favorite_food"],
)

[SemanticFeature(set_id='mem_user_user_a_profile', category='personal_profile', tag='food_and_cuisine', feature_name='favorite_pizza', value='peperoni_pizza', metadata=Metadata(citations=None, id=2, other=None))]

### Updating a memory in your profile

In [38]:
await semantic_session_manager.update_feature(
    feature_a_id,
    value="bbq_pizza",
)

await semantic_session_manager.get_feature(feature_a_id)

SemanticFeature(set_id='mem_user_user_a_profile', category='personal_profile', tag='food_and_cuisine', feature_name='favorite_pizza', value='bbq_pizza', metadata=Metadata(citations=None, id=2, other=None))

### Deleting a memory in your profile

In [39]:
await semantic_session_manager.delete_features(
    feature_ids=[feature_a_id],
)

### Deleteing memories in your profile based on filter

In [40]:
await semantic_session_manager.delete_feature_set(
    user_a_session,
    memory_type=[IsolationType.USER, IsolationType.SESSION],
    tags=["credit_card"],
)

## Adding messages

In order to have your user profile be updated using our LLM ingestion pipeline, raw messages can be provided alongside the isolation levels that will be allowed to process them.

In [41]:
await semantic_session_manager.add_message(
    message="I'm user-a, I really like pizza. Especially peperoni.",
    session_data=user_a_session,
    memory_type=[IsolationType.USER],
)

2

### Checking if a message has been ingested

Messages are analyized and processed into profiles in the background.

Usually the user shouldn't be aware of the details of this process, but it can be useful to check if messages have been processed.

In [42]:
await semantic_session_manager.number_of_uningested_messages(
    session_data=user_a_session,
    memory_type=[IsolationType.USER],
)

2